### Setup

In [1]:
import logging

from delta import DeltaTable

from pyspark.sql import DataFrame
from pyspark.sql.window import Window
import pyspark.sql.functions as F
import pyspark.sql.types as T

StatementMeta(, 6e8f7747-5c01-4ca3-a428-b2604e92d3e4, 3, Finished, Available, Finished)

In [2]:
%run Helpers

StatementMeta(, 6e8f7747-5c01-4ca3-a428-b2604e92d3e4, 5, Finished, Available, Finished)

In [3]:
logger = setup_logger()

StatementMeta(, 6e8f7747-5c01-4ca3-a428-b2604e92d3e4, 6, Finished, Available, Finished)

In [115]:
PLAYLISTS_TABLE = "abfss://fd12376e-2797-4027-bb8e-42a3a8228a70@onelake.dfs.fabric.microsoft.com/1ead427c-19d4-417e-bb8f-a68d9adc0f38/Tables/playlists"
VIDEOS_TABLE = "abfss://fd12376e-2797-4027-bb8e-42a3a8228a70@onelake.dfs.fabric.microsoft.com/1ead427c-19d4-417e-bb8f-a68d9adc0f38/Tables/videos"

RESULTS_TABLE = "abfss://fd12376e-2797-4027-bb8e-42a3a8228a70@onelake.dfs.fabric.microsoft.com/77b89b44-1bcf-42fa-a9ac-7d0593123d3d/Tables/results"
ENHANCED_RESULTS_TABLE = "abfss://fd12376e-2797-4027-bb8e-42a3a8228a70@onelake.dfs.fabric.microsoft.com/1ead427c-19d4-417e-bb8f-a68d9adc0f38/Tables/results"

StatementMeta(, 0dcb56a5-c98e-446a-b888-c13f88c0d791, 117, Finished, Available, Finished)

### Load data

In [162]:
logger.info("Loading playlists and videos")
playlists_df = spark.read.format("delta").load(PLAYLISTS_TABLE)
videos_df = spark.read.format("delta").load(VIDEOS_TABLE)

logger.info("Joining videos and playlists")
videos_with_year = videos_df\
    .select("id","country","playlistId")\
    .join(playlists_df,videos_df.playlistId == playlists_df.PlaylistId, "left")\
    .select(
        F.col("id").alias("video_id"),
        F.col("country").alias("vid_country"),
        F.col("Year").alias("vid_year")
    ).cache()
videos_with_year.count()  # Materialize the cache

StatementMeta(, 0dcb56a5-c98e-446a-b888-c13f88c0d791, 171, Finished, Available, Finished)

2025-03-09 23:36:37,343 - INFO - Loading playlists and videos
2025-03-09 23:36:37,712 - INFO - Joining videos and playlists


201

In [164]:
logger.info("Loading results")
results_df = spark.read.format("delta").load(RESULTS_TABLE)

logger.info(f"results_df loaded with {results_df.count()} rows")
# Ensuring both results and videos use Czechia
results_df = results_df.withColumn("country", F.regexp_replace(F.col("country"), "Czech Republic", "Czechia"))

# Remove notes references 

columns_to_clean = ["country", "artist", "song", "place", "sf_place", "points", "sf_points"]
for column in columns_to_clean:
    results_df = results_df.withColumn(column,F.trim(F.regexp_replace(F.col(column), r"\[.*?\]", "")))

results_df = results_df.withColumn("place", F.when(F.col("place") == "—", None).otherwise(F.col("place"))).withColumn("points", F.when(F.col("points") == "—", None).otherwise(F.col("points")))

# Define a window partitioned by year to compute maximum final placement ("place").
year_window = Window.partitionBy("year")

# Define a window over rows (per year) ordering by sf_points descending.
window_spec = Window.partitionBy("year").orderBy(F.col("sf_points").desc())

# Create the new "new_place" column:
# - When "place" is non-null, use it directly.
# - Otherwise, compute new_place as: COALESCE(max(place) over year, 0) + row_number over the rank_window.
results_df = results_df.withColumn("max_final", F.max(F.col("place")).over(year_window)).withColumn("row_num", F.row_number().over(window_spec)) \
    .withColumn(
        "non_null_count", 
        F.coalesce(
            F.sum(F.when(F.col("place").isNotNull(), F.lit(1))).over(window_spec.rowsBetween(Window.unboundedPreceding, -1)),
            F.lit(0)
        )
    ) \
    .withColumn(
        "all_rank", 
        F.when(F.col("place").isNull(), F.col("max_final") + F.col("row_num") - F.col("non_null_count")).otherwise(F.col("place")).cast(T.IntegerType())
    )\
    .drop("max_final","row_num","non_null_count")

columns_to_cast = [ "place", "sf_place", "points", "sf_points"]
for column in columns_to_cast:
    results_df = results_df.withColumn(column, F.col(column).cast(T.IntegerType()))

logger.info(f"results_df finalised with {results_df.count()} rows")

logger.info("Joining videos and results")
videos_with_year = videos_with_year.dropDuplicates(["vid_year","vid_country"])
output_df = results_df\
    .join(videos_with_year, [results_df.year == videos_with_year.vid_year,results_df.country == videos_with_year.vid_country], "left_outer")\
    .select(
        "year",
        "country",
        "artist",
        "song",
        "video_id",
        "place",
        "points",
        "semi_final",
        "sf_points",
        "sf_place",
        "all_rank",
        "_created_date",
        "_modified_date"
    )\
    .withColumn("video_id", F.coalesce(F.col("video_id"), F.lit("Unknown"))).cache()


logger.info(f"results_df joined with {output_df.count()} rows")
output_df.count()  # Materialize the cache


StatementMeta(, 0dcb56a5-c98e-446a-b888-c13f88c0d791, 173, Finished, Available, Finished)

2025-03-09 23:36:45,853 - INFO - Loading results
2025-03-09 23:36:46,233 - INFO - results_df loaded with 231 rows
2025-03-09 23:36:46,572 - INFO - results_df finalised with 231 rows
2025-03-09 23:36:46,573 - INFO - Joining videos and results
2025-03-09 23:36:46,878 - INFO - results_df joined with 231 rows


231

### Quality check data

To Do

- Build system for no video



### Merge data

In [156]:
def check_table_and_create_if_not_exists(table_path: str, schema: T.StructType) -> None:
    """
    Checks if a Delta table exists at table_path. If not, creates an empty Delta table with the specified schema.
    """
    directory, table_name = table_path.rsplit('/', 1)
    all_databases = spark.catalog.listDatabases()
    database = [t for t in all_databases if t.locationUri == directory][0].name
    logger.debug(f"Checking table {database}.{table_name}")
    if not spark.catalog.tableExists(f"`{database}`.`{table_name}`"):
        logger.info("Table doesn't exists, starting to create")
        df = spark.createDataFrame([], schema)
        df.write.format("delta").save(table_path)
        logger.info("Table creation finished")
    else:
        logger.debug("Table already exists")
    return

StatementMeta(, 0dcb56a5-c98e-446a-b888-c13f88c0d791, 163, Finished, Available, Finished)

In [157]:
def merge_results_data(union_df: DataFrame, table_path: str) -> None:
    """
    Merge the union_df into the Delta table at table_path using country, year, and song as keys.
    Only update rows if any non-key values have changed, and insert any new ones.

    Args:
        union_df: The DataFrame containing the new video data and details.
        table_path: The Delta table ABFS path to merge into.
    """
    try:
        check_table_and_create_if_not_exists(table_path, union_df.schema)

        target_table = DeltaTable.forPath(spark, table_path)
        logger.info("Merging data started")
        target_table.alias("target").merge(
            union_df.alias("source"),
            """
            target.country = source.country
            AND target.year = source.year
            """
        ).whenMatchedUpdate(
            set={
                "artist": "source.artist",
                "song": "source.song",
                "video_id": "source.video_id",
                "place": "source.place",
                "points": "source.points",
                "semi_final": "source.semi_final",
                "sf_points": "source.sf_points",
                "sf_place": "source.sf_place",
                "all_rank": "source.all_rank",
                "_modified_date": "current_timestamp()"
            }
        ).whenNotMatchedInsert(
            values={
                "year": "source.year",
                "country": "source.country",
                "artist": "source.artist",
                "song": "source.song",
                "video_id": "source.video_id",
                "place": "source.place",
                "points": "source.points",
                "semi_final": "source.semi_final",
                "sf_points": "source.sf_points",
                "sf_place": "source.sf_place",
                "all_rank": "source.all_rank",
                "_created_date": "current_timestamp()",
                "_modified_date": "current_timestamp()"
            }
        ).execute()
        logger.info("Merging data finished")
        lastCommit = target_table.history(1).collect()[0]
        metrics = lastCommit["operationMetrics"] 

        numInserted = int(metrics.get("numTargetRowsInserted", 0))
        numUpdated = int(metrics.get("numTargetRowsUpdated", 0))
        numDeleted = int(metrics.get("numTargetRowsDeleted", 0))

        logger.info(f"Rows inserted: {numInserted}")
        logger.info(f"Rows updated: {numUpdated}")
        logger.info(f"Rows deleted: {numDeleted}")
        try:
            logger.info("Start optimize")
            target_table.optimize().executeCompaction()
            logger.info("Finished optimize")
        except Exception as e:
            logger.error("Failed to optimize")
            raise

    except Exception as e:
        logger.exception(f"Merge exception: {str(e)}")
        raise
        


StatementMeta(, 0dcb56a5-c98e-446a-b888-c13f88c0d791, 164, Finished, Available, Finished)

In [160]:
merge_results_data(output_df,ENHANCED_RESULTS_TABLE)

StatementMeta(, 0dcb56a5-c98e-446a-b888-c13f88c0d791, 167, Finished, Available, Finished)

2025-03-09 23:31:00,512 - INFO - Table doesn't exists, starting to create
2025-03-09 23:31:01,533 - INFO - Table creation finished
2025-03-09 23:31:01,689 - INFO - Merging data started
2025-03-09 23:31:04,084 - INFO - Merging data finished
2025-03-09 23:31:04,291 - INFO - Rows inserted: 231
2025-03-09 23:31:04,292 - INFO - Rows updated: 0
2025-03-09 23:31:04,292 - INFO - Rows deleted: 0
2025-03-09 23:31:04,293 - INFO - Start optimize
2025-03-09 23:31:05,436 - INFO - Finished optimize
